In [1]:
import psycopg2

from dotenv import load_dotenv
from os import getenv
import pandas as pd
import numpy as np


In [2]:
load_dotenv()

True

In [ ]:
def upsert_data (df, unique_col, columns_list, table, cur, conn):
    cols = ", ".join(columns_list)
    placeholders = ", ".join(["%s"] * len(columns_list))
    
    # gera lista de updates do tipo col = EXCLUDED.col (exceto a coluna única)
    excluded = ", ".join([
        f"{col} = EXCLUDED.{col}" for col in columns_list if col != unique_col
    ])

    for row in df.itertuples(index=False):
        values = [getattr(row, col) for col in columns_list]
        query = f"""
        INSERT INTO {table} ({cols}) 
        VALUES ({placeholders})
        ON CONFLICT ({unique_col}) DO UPDATE
        SET {excluded};
        """
        try:
            cur.execute(query, values)
        except Exception as e:
            print(f"Erro ao inserir {values}: {e}")
            conn.rollback()
        else:
            conn.commit()

In [4]:
# Estabelecendo conexão com os databases
try:
    first_conn = psycopg2.connect(
        getenv("FIRST_DATABASE_ACCESS")
    )
    first_cur = first_conn.cursor()
    
    second_conn = psycopg2.connect(
        getenv("SECOND_DATABASE_ACCESS")
    )
    second_cur = second_conn.cursor()
except Exception as e:
    print(e)

In [5]:
# Extraindo tabelas do banco da primeira série
try:    
    df_admin_first = pd.read_sql("""
        SELECT 
            nome_admin AS name,
            email,
            senha AS password
        FROM admin
        """,
            first_conn
        )
      
    df_enterprise_first = pd.read_sql("""
        SELECT 
            c.cnpj,
            c.email,
            c.senha AS password,
            c.nome AS name,
            MAX(t.num_telefone) AS phone_number,
            c.id_endereco AS address_id,
            COALESCE(MAX(i.id_plano), 1) AS plan_id
        FROM cliente c
        JOIN telefone t ON t.id_cliente = c.id
        LEFT JOIN industria i ON i.id_cliente = c.id
        GROUP BY c.id, c.cnpj, c.email, password, name, address_id
        """, 
            first_conn
        )

    df_address_first = pd.read_sql("""
        SELECT 
            cep,
            cep_cidade AS city,
            cep_bairro AS neighbourhood,
            cep_rua_num AS house_number,
            cep_estado AS state,
            cep_rua AS public_place,
            cep_complemento AS complement
        FROM endereco
        """,
        first_conn)

    df_employee_first = pd.read_sql("""
        SELECT 
            nome AS name,
            email,
            senha AS password,
            id_empresa AS enterprise_id,
            COALESCE(is_admin, false) AS is_admin
        FROM funcionario
        WHERE id_empresa is not null
        """,
        first_conn)
    
except Exception as e:
    print("Erro:",e)

C:\Users\gabrielbento-ieg\AppData\Local\Temp\ipykernel_11152\153038831.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_admin_first = pd.read_sql("""
C:\Users\gabrielbento-ieg\AppData\Local\Temp\ipykernel_11152\153038831.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_enterprise_first = pd.read_sql("""
C:\Users\gabrielbento-ieg\AppData\Local\Temp\ipykernel_11152\153038831.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_address_first = pd.read_sql("""
C:\Users\gabrielbento-ieg\AppData\Local\Temp\ipykern

In [18]:
df_employee_first

,name,email,password,enterprise_id,is_admin
0,Mariana Oliveira,mariana@empresa1.com,633687b0c0a44b85af838c859eb7a4f6e733ad604b8b27...,10,False
1,Carlos Silva,carlos@empresa1.com,55a5e9e78207b4df8699d60886fa070079463547b095d1...,9,True
2,Rafael Nogueira,rafael.nogueira@gmaio.com,675bf690d55e2a543a85e3c0999d0ce0e0c737e61c84d2...,9,False
3,Beatriz Andrade,beatriz.andrade@gmail.com,d2f3dcf179934a213f2747a7dadc75b41a9fce6329b9af...,9,False


In [7]:
df_admin_first['write'] = False
df_admin_first['image'] = None

# Inserção de imagem genérica para inserção no banco da segunda série
default_image = "https://res.cloudinary.com/dxdjsvo0e/image/upload/v1761314229/f682575d-3083-45b2-a065-f043cf789d64_aonwbs.jpg"

df_enterprise_first["enterprise_image"] = default_image

In [ ]:
# admin insert
upsert_data(df_admin_first, 'email', ['name', 'email', 'password', 'write'], 'admin_saveit', second_cur, second_conn)

# enterprise insert
upsert_data(df_enterprise_first, 'cnpj', ['cnpj', 'email', 'password', 'name', 'phone_number', 'address_id', 'plan_id', 'enterprise_image'], 'enterprise', second_cur, second_conn)

# address insert
upsert_data(df_address_first, 'id', ['cep', 'city', 'neighbourhood', 'house_number', 'state', 'public_place', 'complement'], 'address', second_cur, second_conn)

# employee insert
upsert_data(df_employee_first, 'email', ['name', 'email', 'password', 'enterprise_id', 'is_admin'], 'employee', second_cur, second_conn)